In [12]:
import numpy as np
import pandas as pd
import datetime
import pickle
import os

from sklearn.utils import class_weight
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.utils import to_categorical

In [13]:
root_path = '.'

## Google Colab

In [14]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    root_path = '/content/drive/MyDrive/projects/facial_feeling_detection'
except:
    pass

##### Check GPU

In [15]:
try:
    from tensorflow.python.client import device_lib

    device_lib.list_local_devices()
except:
    pass

## Constants

In [16]:
SEED = 42
LABELS = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
TRAIN_PATH = root_path + '/data/train/'
TEST_PATH = root_path + '/data/test/'
MODELS_PATH = root_path + '/models/'
SUBMISSIONS_PATH = root_path + '/submissions/'

## Data Loading

In [17]:
data = np.load(root_path + '/data/unbalanced_data.npz')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']

with open(root_path + '/data/test_ids.pkl', 'rb') as f:
    test_ids = pickle.load(f)

submission_ids = pd.read_csv(root_path + '/data/test_set.csv')

## Model

In [18]:
learning_rate = .001 # .01
dropout_conv = .3
dropout_dense = .5

In [19]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_conv),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_conv),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_conv),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(dropout_conv),

    Flatten(),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(dropout_dense),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(dropout_dense),
    
    Dense(7, activation='softmax')
])

In [20]:
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# print(model.summary())

In [21]:
callback_path = MODELS_PATH + 'callback.h5'

earlystop = EarlyStopping(patience=7)
mcheckpoint = ModelCheckpoint(
    callback_path,
    monitor='val_accuracy',
    save_best_only=True
)

##### Weights

In [22]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {label: weight for label, weight in zip(np.unique(y_train), class_weights)}

##### To Categorical

In [23]:
y_train = to_categorical(y_train, num_classes=7)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=.2,
    epochs=1,
    batch_size=96,
    class_weight=class_weight_dict,
    # callbacks=[earlystop, mcheckpoint]
    callbacks=[mcheckpoint]
)

In [ ]:
max_val_accuracy = max(history.history['val_accuracy'])
date = datetime.datetime.now().strftime('%y%m%d%H%M%S')
model_path = f'{MODELS_PATH}model_val_acc_{max_val_accuracy:.4f}_{date}.h5'

os.rename(callback_path, model_path)

## Prediction/Submission

In [ ]:
# model_path = MODELS_PATH + 'model_val_acc_0.4901_230224172910.h5'

model_last = load_model(model_path)

In [ ]:
y_pred = model_last.predict(X_test)
predictions = [np.argmax(pred) for pred in y_pred]

221/221 [==============================] - 1s 3ms/step


In [ ]:
submission_pred = pd.DataFrame(test_ids, columns=['id_img']).astype(int)
submission_pred['label'] = [LABELS[pred] for pred in predictions]
submission_pred = pd.merge(submission_ids, submission_pred)

In [ ]:
submission_path = model_path.replace(MODELS_PATH, SUBMISSIONS_PATH).replace('model_', 'submission_').replace('.h5', '.csv')
submission_pred.to_csv(submission_path, index=False)